# Climate Change

## Explore AI Classification Predict

In [34]:
import pandas as pd
import numpy as np
import nltk
import string
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

from sklearn.metrics import f1_score

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
test = pd.read_csv('test.csv') ## ../input/edsa-climate-change-belief-analysis-2021/
train = pd.read_csv('train.csv') ## ../input/edsa-climate-change-belief-analysis-2021/

In [4]:
train.head()

,sentiment,message,tweetid
0,1,PolySciMajor EPA chief doesn't think carbon di...,625221
1,1,It's not like we lack evidence of anthropogeni...,126103
2,2,RT @RawStory: Researchers say we have three ye...,698562
3,1,#TodayinMaker# WIRED : 2016 was a pivotal year...,573736
4,1,"RT @SoyNovioDeTodas: It's 2016, and a racist, ...",466954


Class Description

2 News: the tweet links to factual news about climate change

1 Pro: the tweet supports the belief of man-made climate change

0 Neutral: the tweet neither supports nor refutes the belief of man-made climate change

-1 Anti: the tweet does not believe in man-made climate change

In [5]:
train.isnull().sum()

sentiment    0
message      0
tweetid      0
dtype: int64

In [6]:
blanks = []  # start with an empty list

for i,sent,msg,tweet in train.itertuples():  # iterate over the DataFrame
    if type(msg)==str:            # avoid NaN values
        if msg.isspace():         # test 'review' for whitespace
            blanks.append(i)     # add matching index numbers to the list
        
len(blanks)

0

### Remove URL's Users and Hashtags

In [7]:
train['new'] = train['message'].apply(lambda r : re.sub('@[^\s]+','',r))

In [9]:
train['new'] = train['new'].apply(lambda r : re.sub('RT','',r))

In [12]:
train['new'] = train['new'].str.replace("[^a-zA-Z#]", " ")

In [14]:
train['new'] = train['new'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))

In [26]:
train.head()

,sentiment,message,tweetid,new
0,1,polyscimajor chief doesn think carbon dioxid m...,625221,polyscimajor chief doesn think carbon dioxid m...
1,1,like lack evid anthropogen global warm,126103,like lack evid anthropogen global warm
2,2,research have three year climat chang befor la...,698562,research have three year climat chang befor la...
3,1,#todayinmaker# wire pivot year climat chang ht...,573736,#todayinmaker# wire pivot year climat chang ht...
4,1,racist sexist climat chang deni bigot lead pol...,466954,racist sexist climat chang deni bigot lead pol...


In [16]:
train['new'] = train['new'].apply(lambda x: x.split())

In [19]:
from nltk.stem.porter import PorterStemmer

In [20]:
stemmer = PorterStemmer()

In [21]:
train['new'] = train['new'].apply(lambda x: [stemmer.stem(i) for i in x])

In [23]:
train['new'] = train['new'].apply(lambda x : " ".join(x))

In [25]:
train['message'] = train['new']

In [27]:
train['sentiment'].value_counts()

 1    8530
 2    3640
 0    2353
-1    1296
Name: sentiment, dtype: int64

In [28]:
y = train['sentiment']
X = train['message']

In [29]:
vectorizer = TfidfVectorizer(ngram_range=(1,2), min_df=2, stop_words="english")
X_vectorized = vectorizer.fit_transform(X)

In [37]:
X_train,X_test,y_train,y_test = train_test_split(X_vectorized,y,test_size=.3,shuffle=True, stratify=y, random_state=11)

### Random Forest

In [ ]:
rfc = RandomForestClassifier()
rfc.fit(X_train, y_train)
rfc_pred = rfc.predict(X_test)

In [ ]:
f1_score(y_val, rfc_pred, average="macro")

### Logistic Regression

In [31]:
lm_full = LogisticRegression()
lm_full.fit(X_train, y_train)
pred_lm_full = lm_full.predict(X_val)

/Applications/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [32]:
f1_score(y_val, pred_lm_full, average="macro")

0.5782280619322001

In [41]:
X_train

<11073x16818 sparse matrix of type '<class 'numpy.float64'>'
	with 156485 stored elements in Compressed Sparse Row format>

In [42]:
svc2 =  SVC(kernel='linear',gamma=1, C=1.5)
svc2.fit(X_train, y_train)
pred_svc2 = svc2.predict(X_test)
f1_score(y_test, pred_svc2, average="macro")

0.6197995831288379

### Run Test File and Create Outhput

In [ ]:
test['message'] = test['message'].apply(lambda u : re.sub('((www\.[^\s]+)|(https?://[^\s]+))','URL',u))
test['message'] = test['message'].apply(lambda r : re.sub('@[^\s]+','T_USER',r))
test['message'] = test['message'].apply(lambda h : re.sub(r'#([^\s]+)', r'\1',h))

In [ ]:
testx = test['message']
test_vect = vectorizer.transform(testx)

In [ ]:
y_pred = lm_full.predict(test_vect)

In [ ]:
## y_pred = rfc.predict(test_vect)

In [ ]:
test['sentiment'] = y_pred

In [ ]:
test.head()

In [ ]:
test[['tweetid','sentiment']].to_csv('testsubmission.csv', index=False)